## Importar Paquetes

In [14]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# Revisión de calidad de los datos
## Ruta del directorio que contiene los archivos .geojson

In [15]:
DATA_DIR = Path("data/RF_Layers")

assert DATA_DIR.exists(), f"No se encontró la carpeta: {DATA_DIR.resolve()}"
print("Directorio de trabajo:", DATA_DIR.resolve())

Directorio de trabajo: /Users/sergio/Projects/RedwoodForest/data/RF_Layers


## Localizar los 26 archivos geoJSON

In [16]:
geojson_files = sorted(DATA_DIR.glob("*.geojson"))
print(f"Se encontraron {len(geojson_files)} archivos GeoJSON")
geojson_files[:5]   # vista rápida de los primeros 5

Se encontraron 26 archivos GeoJSON


[PosixPath('data/RF_Layers/Biological_Resources_CDFW_Fishbarriers.geojson'),
 PosixPath('data/RF_Layers/Biological_Resources_Coho_Distribution.geojson'),
 PosixPath('data/RF_Layers/Biological_Resources_Coho_Range.geojson'),
 PosixPath('data/RF_Layers/Biological_Resources_Wildlife_Trees.geojson'),
 PosixPath('data/RF_Layers/California_Data_AdjacentLand.geojson')]

## Función para describir los 26 archivos geoJSON

In [17]:
def describe_layer(path: Path) -> dict:
    gdf = gpd.read_file(path)
    
    crs_text = gdf.crs.to_string() if gdf.crs else "SIN_CRS"
    invalid  = (~gdf.is_valid).sum()      # geometrías no válidas
    geom_types = ", ".join(sorted(gdf.geom_type.unique()))
    
    # lista de atributos excepto la columna geometry
    attrs = ", ".join([c for c in gdf.columns if c != "geometry"])
    
    return {
        "layer": path.stem,
        "features": len(gdf),
        "geom_types": geom_types,
        "crs": crs_text,
        "invalid_geoms": invalid,
        "attributes": attrs
    }

## Generar un inventario con los archivos y su información

In [18]:
inventory = pd.DataFrame([describe_layer(p) for p in geojson_files])
print(f"Inventario generado: {len(inventory)} capas")
inventory.head(6)

Inventario generado: 26 capas


,layer,features,geom_types,crs,invalid_geoms,attributes
0,Biological_Resources_CDFW_Fishbarriers,116,Point,EPSG:4326,0,"OBJECTID, PAD_ID, PassageID, StreamName, Tribu..."
1,Biological_Resources_Coho_Distribution,4650,"LineString, MultiLineString",EPSG:4326,0,"OBJECTID, Early_Yr, Late_Yr, GNIS_ID, GNIS_Nam..."
2,Biological_Resources_Coho_Range,676,Polygon,EPSG:4326,0,"OBJECTID, CALWNUM, CDFPWSName, CDFNUM22, Acrea..."
3,Biological_Resources_Wildlife_Trees,29,Point,EPSG:4326,0,"OBJECTID_1, OBJECTID, Tree_ID, Tree_Speci, DBH..."
4,California_Data_AdjacentLand,304,"MultiPolygon, Polygon",EPSG:4326,3,"OBJECTID, Gross_Acre, Owner_Code, Owner_Name, ..."
5,California_Data_Buildings,257,Point,EPSG:4326,0,"OBJECTID, UNIQUEID, POINT_TYPE, MATERIAL, LOCA..."


## Comprobación de SRCs y validez de geometrías

In [19]:
# 1. ¿Todos los CRS ya están en WGS 84?
all_wgs84 = inventory["crs"].eq("EPSG:4326").all()
print("¿Todos los CRS son EPSG:4326? →", all_wgs84)

# 2. Capas con CRS distinto
bad_crs = inventory[~inventory["crs"].eq("EPSG:4326")]
# 3. Capas con geometrías inválidas
invalids = inventory[inventory["invalid_geoms"] > 0]

display(bad_crs)
display(invalids)

¿Todos los CRS son EPSG:4326? → True


,layer,features,geom_types,crs,invalid_geoms,attributes


,layer,features,geom_types,crs,invalid_geoms,attributes
4,California_Data_AdjacentLand,304,"MultiPolygon, Polygon",EPSG:4326,3,"OBJECTID, Gross_Acre, Owner_Code, Owner_Name, ..."
13,California_Data_Mendo_county_parcels_2013,61010,"MultiPolygon, Polygon",EPSG:4326,27,"OBJECTID, APN, BOOK, PAGE, BLOCK, PARCEL, MULT..."
15,California_Data_Miscelllaneous_Polygons,9,Polygon,EPSG:4326,1,"OBJECTID, UNIQUE_ID, TYPE, DESCRIPTIO, CHGUSER..."


## Corregir geometrías inválidas

In [20]:
from shapely.validation import make_valid   # requiere shapely ≥ 2.0

FIX_DIR = Path("data/FR_Layers_clean")
FIX_DIR.mkdir(exist_ok=True)

for p in geojson_files:
    gdf = gpd.read_file(p)
    
    # Reproyectar si hiciera falta 
    if gdf.crs and gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(4326)

    # Arreglar geometrías inválidas
    if (~gdf.is_valid).any():
        # make_valid es más seguro que buffer(0) en shapely 2.0
        gdf["geometry"] = gdf["geometry"].apply(
            lambda geom: make_valid(geom) if not geom.is_valid else geom
        )

    gdf.to_file(FIX_DIR / p.name, driver="GeoJSON")

print("Capas corregidas guardadas en:", FIX_DIR.resolve())

Capas corregidas guardadas en: /Users/sergio/Projects/RedwoodForest/data/FR_Layers_clean


## Función para describir archivos corregidos
Es similar a la anterior, pero evita errores cuando el resultado es None

In [21]:
def describe_fixed_layer(path: Path) -> dict:
    gdf = gpd.read_file(path)

    crs_text = gdf.crs.to_string() if gdf.crs else "SIN_CRS"
    invalid  = (~gdf.is_valid).sum()

    # Filtra valores None antes de ordenar
    geom_types_list = [gt for gt in gdf.geom_type.unique() if gt is not None]
    geom_types = ", ".join(sorted(map(str, geom_types_list)))

    attrs = ", ".join([c for c in gdf.columns if c != "geometry"])

    return {
        "layer": path.stem,
        "features": len(gdf),
        "geom_types": geom_types or "None",
        "crs": crs_text,
        "invalid_geoms": invalid,
        "attributes": attrs
    }

## Revisar archivos corregidos

In [23]:
geojson_files_fixed = sorted(FIX_DIR.glob("*.geojson"))
inventory_fixed = pd.DataFrame([describe_fixed_layer(p) for p in geojson_files_fixed])

# 1. ¿Todos los CRS ya están en WGS 84?
all_wgs84_fixed = inventory_fixed["crs"].eq("EPSG:4326").all()
print("¿Todos los CRS son EPSG:4326? →", all_wgs84_fixed)

# 2. Capas con CRS distinto
bad_crs_fixed = inventory_fixed[~inventory_fixed["crs"].eq("EPSG:4326")]
# 3. Capas con geometrías inválidas
invalids_fixed = inventory_fixed[inventory_fixed["invalid_geoms"] > 0]

display(bad_crs_fixed)
display(invalids_fixed)


¿Todos los CRS son EPSG:4326? → True


,layer,features,geom_types,crs,invalid_geoms,attributes


,layer,features,geom_types,crs,invalid_geoms,attributes
22,Forestry_Roads,6886,LineString,EPSG:4326,6,"OBJECTID, ROAD_TYPE, SURFACE, STATUS, CONDITIO..."
24,Forestry_Streams,5608,"LineString, MultiLineString",EPSG:4326,5,"OBJECTID, STREAM_TYP, STREAM_COD, CLASS, FLOW,..."


Revisé los archivos con geometrías inválidas en QGIS y no parece que haya pérdidas de información relevantes.

## Soltar (drop) geometrías inválidas de los archivos originales

In [27]:
DEST_DIR = Path("data/FR_Layers_Valid")   # carpeta destino
DEST_DIR.mkdir(exist_ok=True)

def drop_invalid(src_path: Path, dest_dir: Path):
    gdf = gpd.read_file(src_path)
    total = len(gdf)                      # antes de filtrar

    # 1️⃣ Geometrías nulas
    gdf = gdf.dropna(subset=["geometry"])

    # 2️⃣ Geometrías vacías
    gdf = gdf[~gdf.geometry.is_empty]

    # 3️⃣ Geometrías inválidas
    gdf = gdf[gdf.is_valid]

    kept   = len(gdf)
    removed = total - kept

    if kept:
        gdf.to_file(dest_dir / src_path.name, driver="GeoJSON")
        print(f"✔ {src_path.stem}: {kept} válidas · {removed} eliminadas")
    else:
        print(f"⚠ {src_path.stem}: 0 válidas · {removed} eliminadas → no se guardó")

for p in sorted(DATA_DIR.glob("*.geojson")):
    drop_invalid(p, DEST_DIR)

print("\nLimpieza completada. Archivos válidos en:", DEST_DIR.resolve())

✔ Biological_Resources_CDFW_Fishbarriers: 116 válidas · 0 eliminadas
✔ Biological_Resources_Coho_Distribution: 4650 válidas · 0 eliminadas
✔ Biological_Resources_Coho_Range: 676 válidas · 0 eliminadas
✔ Biological_Resources_Wildlife_Trees: 29 válidas · 0 eliminadas
✔ California_Data_AdjacentLand: 301 válidas · 3 eliminadas
✔ California_Data_Buildings: 257 válidas · 0 eliminadas
✔ California_Data_CA_cities: 6904 válidas · 0 eliminadas
✔ California_Data_CA_counties: 58 válidas · 0 eliminadas
✔ California_Data_CA_quad_index: 2882 válidas · 0 eliminadas
✔ California_Data_CalTrans_MilePosts: 311908 válidas · 0 eliminadas
✔ California_Data_CalWater_NorthCoast: 1566 válidas · 0 eliminadas
✔ California_Data_Contour_clipped: 7847 válidas · 0 eliminadas
✔ California_Data_Landmarks: 33 válidas · 0 eliminadas
✔ California_Data_Mendo_county_parcels_2013: 60983 válidas · 27 eliminadas
✔ California_Data_Mendocino_county: 10 válidas · 0 eliminadas
✔ California_Data_Miscelllaneous_Polygons: 8 válidas ·

## Verificación de geometrías resultantes

In [ ]:
DATA_DIR = Path("data/FR_Layers_Valid")   # cambia la ruta
geojson_files_valid = sorted(DATA_DIR.glob("*.geojson"))
inventory_valid = pd.DataFrame([describe_layer(p) for p in geojson_files_valid])
inventory_valid[["layer", "invalid_geoms", "null_geoms"]].head()
